Run `clawpack_wl` and save the figures.

In [8]:
import clawpack_wl as cpa

In [9]:
from clawpack import riemann

import matplotlib.pyplot as plt

# %matplotlib notebook



from clawpack import pyclaw
from clawpack.pyclaw.examples.advection_2d_annulus.mapc2p import mapc2p

import numpy as np
# from clawpack.visclaw import colormaps

import re

Use parts of `run_app_from_main` below (https://github.com/clawpack/pyclaw/blob/151aefa92a613d952f3b49070cd80826a81b1178/src/pyclaw/util.py#L83)

In [10]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# these are usually passed to setup function.
pyclaw_kwargs = {'use_petsc':False,
                'outdir':'./_output',
                'solver_type':'classic'}

In [12]:
clawSolution = cpa.setup()

status = clawSolution.run()
status

2024-05-28 21:41:32,933 INFO CLAW: Solution 0 computed for time t=0.000000
2024-05-28 21:41:33,323 INFO CLAW: Solution 1 computed for time t=0.500000
2024-05-28 21:41:33,703 INFO CLAW: Solution 2 computed for time t=1.000000
2024-05-28 21:41:34,190 INFO CLAW: Solution 3 computed for time t=1.500000
2024-05-28 21:41:34,521 INFO CLAW: Solution 4 computed for time t=2.000000
2024-05-28 21:41:34,825 INFO CLAW: Solution 5 computed for time t=2.500000
2024-05-28 21:41:35,126 INFO CLAW: Solution 6 computed for time t=3.000000
2024-05-28 21:41:35,442 INFO CLAW: Solution 7 computed for time t=3.500000
2024-05-28 21:41:35,740 INFO CLAW: Solution 8 computed for time t=4.000000
2024-05-28 21:41:36,049 INFO CLAW: Solution 9 computed for time t=4.500000
2024-05-28 21:41:36,346 INFO CLAW: Solution 10 computed for time t=5.000000


{'cflmax': 0.4000000000000001,
 'dtmin': 0.013027229415318062,
 'dtmax': 0.013027229415318066,
 'numsteps': 390}

In [13]:
outdir = pyclaw_kwargs.get('outdir','./_output')
htmlplot = pyclaw_kwargs.get('htmlplot',False) # return False if key doesn't exist
iplot    = pyclaw_kwargs.get('iplot',False)
outdir, htmlplot, iplot

('./_output', False, False)

In [14]:
pyclaw.plot.html_plot(outdir=outdir,setplot=cpa.setplot)

Executed setplot successfully
Will plot 11 frames numbered: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Will make 1 figure(s) for each frame, numbered:  [1]

-----------------------------------


Creating html pages for figures...

Directory '/home/ajivani/AdvectionWL/_plots' 
    already exists, files may be overwritten 
Now making png files for all figures...
    Reading  Frame 0 at t = 0  from outdir = /home/ajivani/AdvectionWL/_output
Frame 0 at time t = 0.0
    Reading  Frame 1 at t = 0.5  from outdir = /home/ajivani/AdvectionWL/_output
Frame 1 at time t = 0.5
    Reading  Frame 2 at t = 1  from outdir = /home/ajivani/AdvectionWL/_output
Frame 2 at time t = 1.0
    Reading  Frame 3 at t = 1.5  from outdir = /home/ajivani/AdvectionWL/_output
Frame 3 at time t = 1.5
    Reading  Frame 4 at t = 2  from outdir = /home/ajivani/AdvectionWL/_output
Frame 4 at time t = 2.0
    Reading  Frame 5 at t = 2.5  from outdir = /home/ajivani/AdvectionWL/_output
Frame 5 at time t = 2.5
    Reading  Frame 6 

In [ ]:
r_lower = 3.903302085636277
r_upper = 23.465031329617336
m_r = 126

theta_lower = 0.0
theta_upper = np.pi*2.0
m_theta = 512

r     = pyclaw.Dimension(r_lower,r_upper,m_r,name='r')
theta = pyclaw.Dimension(theta_lower,theta_upper,m_theta,name='theta')
domain = pyclaw.Domain([r,theta])
domain.grid.mapc2p = cpa.mapc2p_annulus
#     domain.grid.num_ghost = solver.num_ghost
domain.grid.num_ghost = 2

In [ ]:
R_nodes,Theta_nodes = domain.grid.p_nodes_with_ghost(num_ghost=2)

In [ ]:
mx = R_nodes.shape[0]-1
my = R_nodes.shape[1]-1
mx, my

In [ ]:
R_nodes.shape

In [ ]:
R_nodes.min(), R_nodes.max()

In [ ]:
# Bottom-left corners
Xp0 = R_nodes[:mx,:my]
Yp0 = Theta_nodes[:mx,:my]

# Top-left corners
Xp1 = R_nodes[:mx,1:]
Yp1 = Theta_nodes[:mx,1:]

# Top-right corners
Xp2 = R_nodes[1:,1:]
Yp2 = Theta_nodes[1:,1:]

# Top-left corners
Xp3 = R_nodes[1:,:my]
Yp3 = Theta_nodes[1:,:my]

In [ ]:
Xp0, Xp1

In [ ]:
Xp2, Xp3

In [ ]:
Xp0.shape

In [ ]:
dx, dy = domain.grid.delta
dx, dy

In [ ]:
def stream_constant(Xp,Yp):
    """ 
    Calculates the stream function in physical space
    """
    return np.pi*(Xp*Yp)

In [ ]:
# Compute velocity component
aux = np.empty((2,mx,my), order='F')

aux[0,:mx,:my] = (stream_constant(Xp1,Yp1)- stream_constant(Xp0,Yp0))/dy
aux[1,:mx,:my] = -(stream_constant(Xp3,Yp3)- stream_constant(Xp0, Yp0))/dx

In [ ]:
print(aux[0, :, :].min(), aux[0, :, :].max())
print(aux[1, :, :].min(), aux[1, :, :].max())

In [ ]:
uux = 1.5 * Xp0
uuy = -1.5 * Yp0

In [ ]:
R_nodes.shape

In [ ]:
fig, ax = plt.subplots()
# levels = np.linspace(data.min(), data.max(), 20)  # Adjust the number of levels as needed
# contour = ax.pcolormesh(xxx, yyy, data, shading='auto')
contour = ax.pcolormesh(R_nodes[:mx,:my], Theta_nodes[:mx, :my], np.sqrt(aux[1, :, :]**2 + aux[0, :, :]**2), 
                        shading='auto')
fig.colorbar(contour)
ax.set_aspect('equal')  # Ensure equal aspect ratio for circles
ax.set_title('Data on Cartesian Grid')
fig

In [ ]:
fig, ax = plt.subplots()
# levels = np.linspace(data.min(), data.max(), 20)  # Adjust the number of levels as needed
# contour = ax.pcolormesh(xxx, yyy, data, shading='auto')
contour = ax.pcolormesh(R_nodes[:mx,:my], Theta_nodes[:mx, :my], np.sqrt(uux**2 + uuy**2), 
                        shading='auto')
fig.colorbar(contour)
ax.set_aspect('equal')  # Ensure equal aspect ratio for circles
ax.set_title('Data on Cartesian Grid')
fig

In [ ]:
plt.scatter(Xp0[1:50, 1:50], Yp0[1:50, 1:50], c='r')
plt.scatter(Xp1[1:50, 1:50], Yp1[1:50, 1:50], c='b')

In [ ]:
plt.scatter(Xp2[1:50, 1:50], Yp2[1:50, 1:50], c='r')
plt.scatter(Xp3[1:50, 1:50], Yp3[1:50, 1:50], c='b')

In [ ]:
plt.scatter(Xp3[1:50, 1:50] - Xp0[1:50, 1:50], Yp3[1:50, 1:50] - Yp0[1:50, 1:50], c='b')
plt.scatter(Xp1[1:50, 1:50] - Xp2[1:50, 1:50], Yp1[1:50, 1:50] - Yp2[1:50, 1:50], c='r')

In [ ]:
plt.scatter(Xp3[1:50, 1:50] - Xp2[1:50, 1:50], Yp3[1:50, 1:50] - Yp2[1:50, 1:50], c='b')
plt.scatter(Xp1[1:50, 1:50] - Xp0[1:50, 1:50], Yp1[1:50, 1:50] - Yp0[1:50, 1:50], c='r')

In [ ]:
# domain.grid.p_nodes_with_ghost(2)[0].min(), domain.grid.p_nodes_with_ghost(2)[0].max()
# domain.grid.p_nodes_with_ghost(2)[1].min(), domain.grid.p_nodes_with_ghost(2)[1].max()

In [ ]:
# domain.grid.p_nodes_with_ghost(2)[0].min(), domain.grid.p_nodes_with_ghost(2)[0].max()
# domain.grid.c_nodes_with_ghost(2)[1].min(), domain.grid.c_nodes_with_ghost(2)[1].max()

In [ ]:
# pyclaw.BC.extrap